In [3]:
# This file will generate the percent change of housing data between 2019 and 2020
!pip install altair

from __future__ import print_function
import sys
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import weekofyear
from pyspark.sql import functions as f
from pyspark.sql.functions import concat_ws
from pyspark.sql.types import StringType

spark = SparkSession.builder.appName("Pct_Chg_Hotness-Max").getOrCreate()

In [7]:
hotness = spark.read.format('csv').options(header='true', inferschema='true').load("NYC_Hotness.csv")
#hotness.show()
hotness = hotness.select(hotness["month_date_yyyymm"].cast(StringType()),"postal_code","zip_name","Area","hotness_score","supply_score","demand_score")

In [15]:
avg_hotness = hotness.withColumn("year", f.substring("month_date_yyyymm", 0, 4)).withColumn("month", f.substring("month_date_yyyymm", 5, 2))
avg_hotness_19 = avg_hotness.where(avg_hotness["year"] == '2019').drop("zip_name","Area").withColumnRenamed("hotness_score","hotness_score_19").withColumnRenamed("supply_score","supply_score_19").withColumnRenamed("demand_score","demand_score_19").withColumnRenamed("postal_code","postal_code_19")
avg_hotness_20 = avg_hotness.where(avg_hotness["year"] == '2020')
#.groupBy('postal_code', 'month_date_yyyymm').mean('hotness_score','supply_score','demand_score').orderBy('month_date_yyyymm')
avg_hotness_change = avg_hotness_20.join(avg_hotness_19, (avg_hotness_20.postal_code == avg_hotness_19.postal_code_19) & (avg_hotness_20.month == avg_hotness_19.month))
#.groupBy('postal_code', 'month').
avg_hotness_change = avg_hotness_change.withColumn("Hotness_Percent_Change", ((avg_hotness_change.hotness_score - avg_hotness_change.hotness_score_19)/avg_hotness_change.hotness_score_19)*100)
avg_hotness_change = avg_hotness_change.withColumn("Supply_Percent_Change", ((avg_hotness_change.supply_score - avg_hotness_change.supply_score_19)/avg_hotness_change.supply_score_19)*100)
avg_hotness_change = avg_hotness_change.withColumn("Demand_Percent_Change", ((avg_hotness_change.demand_score - avg_hotness_change.demand_score_19)/avg_hotness_change.demand_score_19)*100)
avg_hotness_change = avg_hotness_change.select("postal_code","Hotness_Percent_Change","Supply_Percent_Change","Demand_Percent_Change","Area")
avg_hotness_change.show()

+-----------+----------------------+---------------------+---------------------+----+
|postal_code|Hotness_Percent_Change|Supply_Percent_Change|Demand_Percent_Change|Area|
+-----------+----------------------+---------------------+---------------------+----+
|      11215|   -27.680285388607363|  -21.085053365724356|   -77.26286972636373| NYC|
|      11367|     47.83644484186249|    70.71378354439058|   -82.44228445359391| NYC|
|      10065|    -40.56519689546726|  -29.465044831464976|   -64.64910342875723| NYC|
|      11209|    -50.26895229669943|   -48.37051654943243|     -63.510343767728| NYC|
|      10458|     -93.3211453222425|   -89.76722060278051|   -97.53241285286921| NYC|
|      11224|    -59.63215781001726|  -59.259326419186564|   -72.11513183518043| NYC|
|      10038|    -49.21820770302426|  -45.388185045928154|    -82.2856912915771| NYC|
|      11368|   -23.964475067905315|   -16.50641135437002|   -82.13538057252389| NYC|
|      11217|    -46.79754526050918|   -46.34025513517

In [16]:
# Generate data files to be used to plot spatially
avg_hotness_change.coalesce(1).write.option("header", True).csv("avg_hotness_change.csv")

In [9]:
spark.stop()